# 유사한 단어 찾기 게임

1. 사전 학습된 모델 또는 적절한 데이터셋을 찾는다
2. 워드 임베딩 모델을 학습시킨다
3. 단어 유사도가 0.8 이상인 A, B를 랜덤 추출한다
4. A, B와 대응되는 C를 추출한다
5. D를 입력받는다

=> 
A:B = C:D 관계에 대응하는 D를 찾는 게임을 만든다.

ex) A:산, B: 바다, C:나무, D:물

**<출력 예시>**

관계 [ 수긍 : 추락 = 대사관 : ?]<br>
모델이 예측한 가장 적합한 단어: 잠입<br>
당신의 답변과 모델 예측의 유사도: 0.34<br>
아쉽네요. 더 생각해보세요.

In [67]:
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-SyntheticText-1.5B")

https://huggingface.co/datasets/HAERAE-HUB/KOREAN-SyntheticText-1.5B

In [68]:
print(ds)

print(ds['train'][0])

DatasetDict({
    train: Dataset({
        features: ['text', '__index_level_0__'],
        num_rows: 1552370
    })
})
{'text': '소녀들을 위한 트윈셋의 특별한 날을 위한 드레스\n\n특별한 날을 위한 트윈셋의 소녀 드레스는 세심한 장인정신, 디테일, 색상, 대담한 프린트로 소녀들의 개성을 표현할 수 있게 해줍니다. 첫 영성체 드레스는 러플, 튈, 주름, 반짝이, 자수 등의 디테일로 현대적인 공주님들을 위해 디자인되었습니다. 재킷과 팬츠, 롱 스커트, 탑, 쇼츠 등 다양한 아이템을 믹스매치하여 특별한 날을 위한 완벽한 스타일을 완성할 수 있습니다. 또한, 풍부한 색상과 정교한 장식을 갖춘 소녀들을 위한 티셔츠, 봄버 자켓, 블레이저를 제공하여 현대적인 감각을 더합니다.\n\n트윈셋의 특별한 날을 위한 소녀 드레스 라인은 아이코닉하고 세련된 니트웨어, 재미있는 액세서리, 그리고 편안하고 우아한 신발로 구성되어 있습니다. 반짝이는 풀 스팽글 드레스는 소녀들의 꿈을 실현시켜 줍니다. 이 컬렉션은 소녀들의 순수함과 우아함을 강조합니다.\n\n이번 챕터에서는 트윈셋의 특별한 날을 위한 소녀 드레스 라인의 미학, 장인정신, 그리고 세심한 디테일에 대해 심층적으로 탐구할 것입니다. 우리는 이러한 드레스들이 어떻게 현대 소녀들의 개성과 스타일을 표현하는지, 그리고 어떻게 특별한 날과 이벤트를 더욱 잊지 못할 순간으로 만들어 주는지 이해할 것입니다.\n\n우선, 트윈셋의 소녀 드레스의 미학은 "소녀들의 우정"을 상징합니다. 이러한 디자인은 소녀들의 순수함, 우정, 그리고 꿈을 반영하며, 그들의 우정과 소중한 순간을 축하합니다. 이를 위해, 디자이너들은 부드럽고 로맨틱한 색상, 그리고 소녀들의 세계를 반영하는 대담하고 재미있는 프린트에 중점을 두었습니다.\n\n또한, 장인정신은 트윈셋의 소녀 드레스의 핵심입니다. 각 드레스는 정교한 디테일과 고급스러운 소재로 만들어집니다. 예를

In [69]:
ds = ds.remove_columns([column for column in ds['train'].column_names if column != 'text'])

In [70]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1552370
    })
})


In [110]:
ds_small = ds['train'].shuffle(seed=42).select(range(50000))

In [111]:
print(ds_small)

Dataset({
    features: ['text'],
    num_rows: 50000
})


In [ ]:
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from konlpy.tag import Okt
from tqdm import tqdm
import random
from gensim.models import FastText
from gensim import matutils

In [112]:
corpus_list = ds_small['text']
corpus_list = [re.sub(r'\([^)]*\)', '', text) for text in corpus_list]
corpus_list = [re.sub(r'[^가-힣\s]', ' ', text) for text in corpus_list]
okt = Okt()
ko_stopwords = ["은", "는", "이", "가", "을", "를", "과", "와", "들", "도", "부터", "까지", "에", "나", "너", "그", "걔", "얘"]

preprocessed_data = []

# for sentence in tqdm(ds_small['text']): 
#     tokens = okt.morphs(sentence, stem=True)  
#     tokens = [token for token in tokens if token not in ko_stopwords]  
#     preprocessed_data.append(tokens)

for sentence in tqdm(corpus_list): 
    tokens = okt.morphs(sentence, stem=True)  
    tokens = [token for token in tokens if token not in ko_stopwords]  
    preprocessed_data.append(tokens)

100%|██████████| 50000/50000 [1:03:58<00:00, 13.03it/s] 


In [113]:
fasttext_model = FastText(
    sentences=preprocessed_data,  
    vector_size=100,              
    window=5,                     
    min_count=5,                  
    sg=1                          
)

In [114]:
fasttext_model.wv.vectors.shape

(35525, 100)

In [115]:
fasttext_model.wv.most_similar('기술')

[('습득', 0.6919926404953003),
 ('생체인식', 0.679587721824646),
 ('사물인터넷', 0.6630621552467346),
 ('인포텍', 0.6559842228889465),
 ('분야', 0.6531525254249573),
 ('혁신', 0.6514034271240234),
 ('지식', 0.6485097408294678),
 ('솔리디티', 0.6402550339698792),
 ('첨단', 0.6366191506385803),
 ('기계학습', 0.6348678469657898)]

In [116]:
fasttext_model.wv.most_similar('감기')

[('독감', 0.7207053303718567),
 ('고열', 0.720640242099762),
 ('기침', 0.6801483035087585),
 ('부비동염', 0.6674129366874695),
 ('기관지염', 0.6619825959205627),
 ('몸살', 0.660027265548706),
 ('머릿니', 0.6574501395225525),
 ('항바이러스제', 0.6564996838569641),
 ('위궤양', 0.6561011672019958),
 ('콩팥', 0.6528971791267395)]

In [117]:
fasttext_model.wv.most_similar('드레스')

[('웨딩드레스', 0.8086466193199158),
 ('스웨터', 0.8003941178321838),
 ('청바지', 0.7885757088661194),
 ('셔츠', 0.7845620512962341),
 ('팬츠', 0.7841965556144714),
 ('정장', 0.7790696024894714),
 ('블라우스', 0.7687807083129883),
 ('튤', 0.7626127004623413),
 ('데님', 0.7619402408599854),
 ('이브닝', 0.7615557312965393)]

In [118]:
vocab = list(fasttext_model.wv.index_to_key)

In [119]:
def random_pair(min = 0.8):
    while True:
        a = random.choice(vocab)
        similar_words = [word for word, sim in fasttext_model.wv.most_similar(a, topn=50) if sim >= min]
        if similar_words:
            b = random.choice(similar_words)
            return a, b #similar_words가 없는 경우에 에러나서 if 넣음...
    
def random_word(a, b):
    while True:
        c = random.choice(vocab)
        if c != a and c != b:
            return c

In [165]:
A, B = random_pair()
C = fasttext_model.wv.most_similar(positive=[A, B], topn=1)[0][0]

In [166]:
print(A, B, C)

기슭 나이아가라 산기슭


In [169]:
D = input(f"관계 [ {A} : {B} = {C} : ?] \n당신의 답: ")

In [168]:
predict_word = fasttext_model.wv.most_similar(positive=[B, C], negative=[A], topn=1)[0][0]
predict_word

'늪지'

In [170]:
vd = fasttext_model.wv[D]
sim = fasttext_model.wv.similarity(D, predict_word)

# 결과 출력
print(f"\n관계 [ {A} : {B} = {C} : ? ]")
print(f"모델이 예측한 가장 적합한 단어: {predict_word}")
print(f"당신이 선택한 단어: {D}")
print(f"당신의 답변과 모델 예측의 유사도: {sim:.2f}")

if predict_word[1] == D:
    print("정답!!!")
else:
    print("아쉽네요. 더 생각해보세요.")


관계 [ 기슭 : 나이아가라 = 산기슭 : ? ]
모델이 예측한 가장 적합한 단어: 늪지
당신이 선택한 단어: 폭포
당신의 답변과 모델 예측의 유사도: 0.69
아쉽네요. 더 생각해보세요.
